In [8]:
#define your parameters here
#used reference from here: https://www.kaggle.com/code/prakharbatwal/transformer
pretrained_model = 'bert-base-uncased'
num_epochs = 3
learning_rate = 5e-5
batch_size = 8

In [9]:
import torch

if(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu"

print(f"using {device} for processing")

using cuda for processing


In [1]:
import pandas as pd;import numpy as np
df = pd.read_csv("/kaggle/input/raw-news/news.csv")
df.drop_duplicates(subset='text', inplace=True)
df.dropna(subset=['text', 'label'], inplace=True)
df = df[df['text'].str.strip() != ""]
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [2]:
from sklearn.model_selection import train_test_split
X = df['text'].tolist()
y = df['label'].tolist()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [3]:
type(y_train)

list

In [4]:
print("Train Label Counts:", pd.Series(y_train).value_counts())


Train Label Counts: 1    16983
0    13932
Name: count, dtype: int64


In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=256,return_tensors="pt").to(device)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=256,return_tensors="pt").to(device)

In [13]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)

In [14]:
train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)
train_dataset.__getitem__(1000)

/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([  101, 13229,  2763, 20010,  4355,  2015,  6926,  8845,  2040, 13950,
          2068,  2006,  2037, 21882,  7088,  5289,  2015,  8275,  2739,  9499,
           999,  3422,  1996,  2678,  1998,  2156,  2129,  1996, 13229,  6443,
          3269,  2048, 12060,  2157,  2279,  2000,  2169,  2060,  2021,  1037,
         28988,  2008, 12271,  2083,  1996,  2915, 13950,  2015,  2068,  2006,
          2037,  3975,  8040,  4402,  2368,  8275,  2739,  2693,  2664,  2178,
         18932,  1997, 21699,  1001,  8275,  2638,  9333,  2004,  1001,  8275,
          2638,  9333,  2278, 10695,  3594,  1037,  3975,  3898,  2005, 12060,
          3061,  2157,  2279,  2000,  2169,  2060,   999,  1001, 23848,  2050,
         27263,  1012, 10474,  1012,  4012,  1013,  1062, 26291, 16257,  5705,
          2100,  5666,  8398, 19454, 24612,  1006,  1030,  4913,  4890,  5737,
          6137,  1007,  2281,  1016,  1010,  2418,  8545,  2310,  3139,  2060,
          8275,  2739,  5312,  2013, 13

In [15]:
for i in range(5):
    sample = train_dataset[i]
    print(f"Label: {sample['labels']}, Decoded: {tokenizer.decode(sample['input_ids'])[:100]}")


Label: 0, Decoded: [CLS] president trump told the graduates that they are graduating to a totally brilliant future. tru
Label: 1, Decoded: [CLS] moscow ( reuters ) - russia designated radio free europe / radio liberty ( rfe / rl ) and voic
Label: 0, Decoded: [CLS] after hillary clinton made the argument that donald trump s hateful rants against muslims were
Label: 1, Decoded: [CLS] new york ( reuters ) - u. s. presidential candidate hillary clinton ’ s spokeswoman said on fr
Label: 1, Decoded: [CLS] harare ( reuters ) - retired army chief constantino chiwenga and veteran politician kembo moha


/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [16]:
print(type(train_encodings))
print(train_encodings.keys())


<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [17]:
sample = train_dataset.__getitem__(1)
print("Keys:", sample.keys())
print("Input IDs:", sample["input_ids"][:10])  # Show first 10 tokens
print("Label:", sample["labels"])
print("Decoded:", tokenizer.decode(sample["input_ids"]))


Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
Input IDs: tensor([  101,  4924,  1006, 26665,  1007,  1011,  3607,  4351,  2557,  2489],
       device='cuda:0')
Label: tensor(1)
Decoded: [CLS] moscow ( reuters ) - russia designated radio free europe / radio liberty ( rfe / rl ) and voice of america ( voa ) as foreign agents on tuesday, a move aimed at complicating their work in retaliation for what moscow says is unacceptable u. s. pressure on russian media. russia s broadside is likely to further sour battered u. s. - russia relations and is part of the fallout from allegations that the kremlin meddled in the u. s. presidential election last year in donald trump s favor, something moscow denies. u. s. intelligence officials accused the kremlin of using russian media it finances to influence u. s. voters, and russian state broadcaster rt last month reluctantly complied with a u. s. request to register a u. s. - based affiliate as a foreign agent under the 

/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [21]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=2).to(device)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch", #saves the model after every epoch into the out directory
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    learning_rate = learning_rate,
    gradient_accumulation_steps=1,
    fp16=True,
    report_to="none",
    dataloader_pin_memory=True if device == "cpu" else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

# Sanity check random entries
print("Sample train label:", y_train[0])
print("Sample test label:", y_test[0])
print("Sample train text:", X_train[0][:300])  # preview first 300 chars
print("Sample test text:", X_test[0][:300])



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train size: 30915
Test size: 7729
Sample train label: 0
Sample test label: 1
Sample train text: President Trump told the graduates that they are  graduating to a totally brilliant future. Trump specifically recognized graduates who have served in the military,  It is truly a testament to this university and to the values that you embrace, that your graduating class includes so many patriots wh
Sample test text: LONDON (Reuters) - A British government official cast doubt on a newspaper report on Tuesday that Britain and the EU had reached a financial settlement of between 45 billion and 55 billion euros.  I do not recognize this account of the negotiations,  said the government official, who declined to be 


In [22]:
model.device

device(type='cuda', index=0)

In [23]:
trainer.train()
trainer.evaluate()

/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.000100,0.001324
2,0.000000,0.001565
3,0.000000,0.001259


/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.001259368029423058,
 'eval_runtime': 56.4399,
 'eval_samples_per_second': 136.942,
 'eval_steps_per_second': 17.133,
 'epoch': 3.0}

In [24]:
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import confusion_matrix


preds = trainer.predict(test_dataset)
print("Raw logits for first 5 samples:\n", preds.predictions[:5])

y_pred = np.argmax(preds.predictions, axis=1)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Raw logits for first 5 samples:
 [[-6.0351562  6.6992188]
 [ 6.25      -6.8789062]
 [ 6.2460938 -6.859375 ]
 [ 6.2226562 -6.9101562]
 [-6.0351562  6.703125 ]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3521
           1       1.00      1.00      1.00      4208

    accuracy                           1.00      7729
   macro avg       1.00      1.00      1.00      7729
weighted avg       1.00      1.00      1.00      7729

[[3521    0]
 [   1 4207]]


In [25]:
import torch

random_texts = ["this is gibberish", "you won a lottery", "election results"]
inputs = tokenizer(random_texts, return_tensors="pt", truncation=True, padding=True)

# Move inputs to model's device (usually cuda:0)
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict
outputs = model(**inputs)
preds = torch.argmax(outputs.logits, axis=1)
print(preds)


tensor([0, 0, 0], device='cuda:0')
